In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')
small_llm = ChatOpenAI(model='gpt-4o-mini')

### 1. RDBMS to VectorDB
- 변동성이 적은 Admin 메뉴 데이터 VeoctorDB 적재

In [3]:
import mysql.connector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="noticeboard"
)
cursor = conn.cursor(dictionary=True)

# 1. 데이터 조회
cursor.execute("""
    SELECT page_id, domain, page_url, description
    FROM page_metadata
""")
rows = cursor.fetchall()

# 2. LangChain 데이터 형식(Document)으로 변환
documents = []
for row in rows:
    # 벡터화할 핵심 텍스트: description
    page_content = row['description'] 
    
    # 저장할 메타데이터: 검색 결과에서 꺼내 쓸 정보들
    metadata = {
        "page_id": row["page_id"],
        "domain": row["domain"],
        "page_url": row["page_url"]
    }
    
    documents.append(Document(page_content=page_content, metadata=metadata))

# 3. Embedding 모델 설정 및 Vector DB 저장
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma DB에 저장
vector_db = Chroma.from_documents(
    documents, 
    embeddings, 
    persist_directory="./my_vector_db"
)


In [4]:
# 사용자의 질문(의도) 시뮬레이션
query = "비밀번호를 잊어버렸는데 로그인 어떻게 해?"

# 유사도 검색 수행 (가장 유사한 2개 가져오기)
search_results = vector_db.similarity_search(query, k=2)

for doc in search_results:
    print(f"도메인: {doc.metadata['domain']}")
    print(f"경로(URL): {doc.metadata['page_url']}")
    print(f"매칭된 설명: {doc.page_content}")
    print("-" * 30)

도메인: 회원
경로(URL): /login
매칭된 설명: 사용자가 아이디와 비밀번호로 로그인하는 화면이다. 인증 성공 시 서비스에 접근할 수 있다.
------------------------------
도메인: 보안
경로(URL): /security/2fa
매칭된 설명: 사용자의 계정 보안을 강화하기 위해 2단계 인증을 설정하는 화면이다. OTP 또는 QR 기반 인증을 설정한다.
------------------------------


### 2. RAG 답변 체인 구성

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 디자인 (페르소나 부여)
template = """
당신은 시스템 이용을 돕는 친절한 AI 가이드입니다. 
사용자의 질문에 대해 [검색 결과]를 바탕으로 자연스럽고 친절하게 답변하세요.
답변에는 반드시 이동해야 할 페이지의 이름과 URL을 포함해 주세요.

[검색 결과]:
{context}

사용자 질문: {question}

친절한 답변:"""

prompt = ChatPromptTemplate.from_template(template)

# 검색 및 답변 생성 체인 구성
def format_docs(docs):
    # 검색된 문서들을 하나의 텍스트로 합침
    return "\n\n".join([f'''페이지명: {d.metadata['domain']}
                        URL: {d.metadata['page_url']}\n설명: {d.page_content}''' for d in docs])

retriever = vector_db.as_retriever(search_kwargs={"k": 2})

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [6]:
response = chain.invoke("재고 보고싶어")
print(response.content)

안녕하세요! 상품의 재고 현황과 입출고 내역을 확인하고 싶으시군요. 재고 관리를 위해서는 "재고" 페이지로 이동하시면 됩니다. 아래의 링크를 클릭하시면 자세한 정보를 확인하실 수 있습니다:

[재고 페이지](/inventory)

필요한 정보를 잘 찾으시길 바랍니다! 추가로 궁금한 점이 있으면 언제든지 말씀해 주세요.
